In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install -r ./drive/MyDrive/NLU\ Project/CODE/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import numpy as np

import transformers
import sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
from datasets import load_dataset
import ast
import time
import json

from tqdm import tqdm


In [ ]:
load_dotenv()
api_key = os.getenv('your-key')
client = OpenAI(api_key='your-key')


In [ ]:
base_dir = "./drive/MyDrive/NLU Project/"
data = pd.read_csv(base_dir + 'DATA/nlu_data.csv')
data

,Unnamed: 0,Question,Context,Answer,Choice_1,Choice_2,Choice_3,Choice_4,Shuffled Context
0,0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,Saint Therese of Lisieux,Saint Bernadette Soubirous,Saint Catherine of Siena,Saint Joan of Arc,"In 2010, a genetic study was conducted on the ..."
1,1,When did the Scholastic Magazine of Notre dame...,"As at most other universities, Notre Dame's st...",September 1876,September 1876,January 1872,March 1880,August 1865,"On November 2, 2014, the Cubs announced that J..."
2,2,Where is the headquarters of the Congregation ...,The university is the major seat of the Congre...,Rome,Athens,Rome,Jerusalem,Mecca,"In higher education, Politecnico refers to a t..."
3,3,How many BS level degrees are offered in the C...,The College of Engineering was established in ...,eight,five,four,eight,two,While Southeast Asia is rich in flora and faun...
4,4,What entity provides help with the management ...,All of Notre Dame's undergraduate students are...,Learning Resource Center,Learning Support Facility,Student Success Hub,Career Development Center,Learning Resource Center,Baird Television Ltd. made Britain's first tel...
...,...,...,...,...,...,...,...,...,...
9995,9995,What was the film that Madonna starred in 2000?,"In 2000, Madonna starred in the film The Next ...",The Next Best Thing,The Next Best Thing,The Last Great Thing,The Perfect Opportunity,The Ultimate Solution,Seattle is considered the home of grunge music...
9996,9996,Who is Madonna's second husband?,"She met director Guy Ritchie, who would become...",Guy Ritchie,Actor Johnson,Director Smith,Guy Ritchie,Producer Brown,The economic reforms of President Carlos Salin...
9997,9997,Which film did Madonna starred in that was dir...,"Madonna starred in the film Swept Away, direct...",Swept Away,Swept Away,Wyndhams Theatre,Die Another Day,Up For Grabs,"Bell's patent 174,465, was issued to Bell on M..."
9998,9998,What is the name of the exhibition installatio...,"Following Die Another Day, Madonna collaborate...",X-STaTIC Pro=CeSS,X-STaTIC Pro=CeSS,X-STiC Pro=CESS,X-CPXTIC Pro=CeSS,X-STaTIX Pro=CeSS,"The most pelagic species, mainly in the 'tuben..."


In [ ]:
# Actually running Experiment 1
questions, real_contexts, shuffled_contexts, correct_answers, choices, correct_context_question_choice_combos, incorrect_context_question_choice_combos = [], [], [], [], [], [], []


total_rows = 1000

for ind in range(total_rows):
  row = data.iloc[ind]
  questions.append(row['Question'])
  real_contexts.append(row['Context'])
  shuffled_contexts.append(row['Shuffled Context'])
  correct_answers.append(row['Answer'])
  choices.append([row['Choice_1'], row['Choice_2'], row['Choice_3'], row['Choice_4']])

  options = {
      'A' : row['Choice_1'],
      'B' : row['Choice_2'],
      'C' : row['Choice_3'],
      'D' : row['Choice_4']
      }
  correct_context_question_choice_combos.append({
      'Context' : row['Context'],
      'Question' : row['Question'],
      'Options' : options})
  incorrect_context_question_choice_combos.append({
      'Context' : row['Shuffled Context'],
      'Question' : row['Question'],
      'Options' : options})

print(len(questions))

1000


In [ ]:
ind = 10
print(f'Question:\n{questions[ind]} \n\n\nReal Context:\n{real_contexts[ind]} \n\n\nShuffled Context:\n {shuffled_contexts[ind]} \n\n\nCorrect Answer:\n{correct_answers[ind]} \n\n\nChoices:\n{choices[ind]} \n\n\nCorrect_context_question_choice_combo:\n{correct_context_question_choice_combos[ind]} \n\n\nIncorrect_context_question_choice_combo:\n{incorrect_context_question_choice_combos[ind]}')
# print()

Question:
What person was the Director of the Science Museum at Notre Dame in the late 19th century? 


Real Context:
Father Joseph Carrier, C.S.C. was Director of the Science Museum and the Library and Professor of Chemistry and Physics until 1874. Carrier taught that scientific research and its promise for progress were not antagonistic to the ideals of intellectual and moral culture endorsed by the Church. One of Carrier's students was Father John Augustine Zahm (1851–1921) who was made Professor and Co-Director of the Science Department at age 23 and by 1900 was a nationally prominent scientist and naturalist. Zahm was active in the Catholic Summer School movement, which introduced Catholic laity to contemporary intellectual issues. His book Evolution and Dogma (1896) defended certain aspects of evolutionary theory as true, and argued, moreover, that even the great Church teachers Thomas Aquinas and Augustine taught something like it. The intervention of Irish American Catholics in

In [ ]:
base_messages = [{"role": "system", "content": """You have a question with four answer choices and a context which may or may not be associated with it. Your task is to return the correct answer option and answer. Reference delimited with ### to outline real examples of user input of contexts, questions, answers, and expected outputs.
######
RESPONSE STRUCTURE:
You must output your response as a JSON object following this structure:
{"thought": {{step by step thoughts to answer the question given the provided context}}, "answer_letter": {{correct answer letter (A, B, C, or D)}}, "answer_name": {{correct answer text}}}
######
EXAMPLES:
###
{"role": "user", "content": "context: A Science Hall was built in 1883 under the direction of Fr. Zahm, but in 1950 it was converted to a student union building and named LaFortune Center, after Joseph LaFortune, an oil executive from Tulsa, Oklahoma. Commonly known as "LaFortune" or "LaFun," it is a 4-story building of 83,000 square feet that provides the Notre Dame community with a meeting place for social, recreational, cultural, and educational activities. LaFortune employs 35 part-time student staff and 29 full-time non-student staff and has an annual budget of $1.2 million. Many businesses, services, and divisions of The Office of Student Affairs are found within. The building also houses restaurants from national restaurant chains.\nquestion: Which person oversaw the creation of a science hall at Notre Dame in 1883?\nanswers: A. Fr. Zahm\nB. Fr. Augustus\tC. St. Geller\tD. Rev. Greene"}
{"role": "assistant", "content": "{"thought": "Let us think step by step. The text states that a Science Hall was built in 1883 under the direction of Fr. Zahm. Thus, Fr. Zahm oversaw the creation of a science hall at Notre Dame in 1883. Therefore, the correct answer is: A. Fr. Zahm.", "answer_letter": "A", "answer_name": "Fr. Zahm"}"}
###
{"role": "user", "content" "context: The New York Islanders and the New York Rangers represent the city in the National Hockey League. Also within the metropolitan area are the New Jersey Devils, who play in nearby Newark, New Jersey.\nquestion: There are two hockey teams located in NYC. What are they?\nanswers: A. The New York Knicks and the Brooklyn Nets\nB. The New York Yankees and the New York Mets\nC. The New York Islanders and the New York Rangers\nD. The New York Giants and the New York Sharks"}
{"role": "assistant", "content": "{"thought": "Let us think step by step. The text states that the New York Islanders and the New York Rangers represent the city in the National Hockey League. Thus, those are the names of the two hockey teams located in NYC. Therefore, the correct answer is: C. The New York Islanders and the New York Rangers.", "answer_letter": "C", "answer_name": "The New York Islanders and the New York Rangers"}"}
###
{"role": "user", "content" "context: Guest judges may occasionally be introduced. In season two, guest judges such as Lionel Richie and Robin Gibb were used, and in season three Donna Summer, Quentin Tarantino and some of the mentors also joined as judges to critique the performances in the final rounds. Guest judges were used in the audition rounds for seasons four, six, nine, and fourteen such as Gene Simmons and LL Cool J in season four, Jewel and Olivia Newton-John in season six, Shania Twain in season eight, Neil Patrick Harris, Avril Lavigne and Katy Perry in season nine, and season eight runner-up, Adam Lambert, in season fourteen.\nquestion: In what season was Lionel Richie a guest judge on American Idol?\nanswers: A. season three\nB. season two\nC. season seven\nD. season six"}
{"role": "assistant", "content": "{"thought": "Let us think step by step. The text states that in ", "answer_letter": "B", "answer_name": "season two", "explanation": "The text states that Lionel Richie was a guest judge in season two."}"}
###
"""}]

def call_api(model_input_args, model_name = "gpt-3.5-turbo"):
    '''
    Args:
    model_input_args : {
      "Context" : Str - context string
      "Question" : Str - Question String
      "Options" : Dict {'A' : Option_1,
                        'B' : Option_2,
                        'C' : Option_3,
                        'D' : Option_4
      }

    Returns:
      {'model_response' : model_response_string}

    '''
    global client, base_messages
    choices = model_input_args['Options']

    messages = base_messages + [{"role": "user", "content": f"""context: {model_input_args['Context']}\nquestion: {model_input_args['Question']}\nanswers: A. {choices['A']}\tB. {choices['B']}\tC. {choices['C']}\tD{choices['D']}"""}]

    attempts = 0
    while attempts < 3:
        try:
            # print(messages[1])
            response = client.chat.completions.create(model=model_name, messages=messages)
            # print('\n\nRESPONSE:\n', response)
            content = response.choices[0].message.content
            # print('\n\nCONTENT:\n', content)
            # print('\n\n\n\n\n\n\n\n\n\n\n\n')
            # content_dict = ast.literal_eval(content)
            # return content_dict
            return {content}
        except SyntaxError as e:
            if e == 'invalid syntax (<unknown>, line 1)':
                print(f"Error for model input \n{model_input_args}\n\n \'{e}\'")
                return {content}
            print(f"Error for model input \n{model_input_args}\n\n {e}")
            attempts += 1
            if attempts < 3:
                print("Retrying in 1 minute...")
                time.sleep(60)
            else:
                print("Failed after 3 attempts.")
                return {content}

In [ ]:
call_api(correct_context_question_choice_combos[0])

{'{"thought": "Let us think step by step. The text mentions that the Virgin Mary allegedly appeared in 1858 in Lourdes, France to Saint Bernadette Soubirous. Therefore, the correct answer is B. Saint Bernadette Soubirous.", "answer_letter": "B", "answer_name": "Saint Bernadette Soubirous"}"}'}

In [ ]:
correct_context_question_choice_combos[0]['Options']

{'A': 'Saint Therese of Lisieux',
 'B': 'Saint Bernadette Soubirous',
 'C': 'Saint Catherine of Siena',
 'D': 'Saint Joan of Arc'}

In [ ]:
correct_answers[0]

'Saint Bernadette Soubirous'

In [ ]:
def run_method_relevant_context(context_question_choice_combos, method = 'relevant'):
    """

    Args:
      context_question_choice_combos: List - [{
                                            "Context" : Str - context string
                                            "Question" : Str - Question String
                                            "Options" : Dict {'A' : Option_1,
                                                              'B' : Option_2,
                                                              'C' : Option_3,
                                                              'D' : Option_4
                                                              }
                                            ]
      method : Method to be run - relevant/ irrelevant/ none

    Returns:

    """
    responses = []
    if method == 'no_context':
        for i in context_question_choice_combos:
          i['Context'] = ''
    for i in tqdm(context_question_choice_combos):
      responses.append(call_api(i))


    return responses

In [ ]:
model_outputs_relevant = run_method_relevant_context(correct_context_question_choice_combos[:20], method = 'relevant')
model_outputs_relevant

100%|██████████| 20/20 [00:39<00:00,  2.00s/it]


[{'{"thought": "Let us think step by step. The text mentions that the Virgin Mary allegedly appeared to Saint Bernadette Soubirous in 1858 in Lourdes, France. Therefore, the correct answer is B. Saint Bernadette Soubirous.", "answer_letter": "B", "answer_name": "Saint Bernadette Soubirous"}"'},
 {'{"thought": "Let us think step by step. The context mentions that The Scholastic magazine began as a one-page journal in September 1876 and it claims to be the oldest continuous collegiate publication in the United States. Therefore, the Scholastic Magazine of Notre Dame began publishing in September 1876.", "answer_letter": "A", "answer_name": "September 1876"}'},
 {'{"thought": "Let us think step by step. The text mentions that the official headquarters of the Congregation of the Holy Cross are in Rome, not in Notre Dame. Therefore, the correct answer is: B. Rome.", "answer_letter": "B", "answer_name": "Rome"}"'},
 {'{"thought": "Let us think step by step. The text states that the College o

In [ ]:
list(model_outputs_relevant[0])[0]

'{"thought": "Let us think step by step. The text mentions that the Virgin Mary allegedly appeared to Saint Bernadette Soubirous in 1858 in Lourdes, France. Therefore, Saint Bernadette Soubirous is the correct answer.", "answer_letter": "B", "answer_name": "Saint Bernadette Soubirous"}"}'

In [ ]:
json_resp = json.loads(list(model_outputs_relevant[0])[0])
print(type(json_resp))

JSONDecodeError: Extra data: line 1 column 289 (char 288)

In [ ]:
json_resp

In [ ]:
def check_match(model_output, options, correct_answer):
  """

  Args:
    model_output: Str - Model prediction according to the method being used
    options: Dict {'A' : Option_1,
                        'B' : Option_2,
                        'C' : Option_3,
                        'D' : Option_4
      }
    correct_answer: Str - Target prediction (Real Answer)

  Returns: True/False to evaluate model correctness

  """
  options = [options['A'], options['B'], options['C'], options['D']]
  correct_answer_index = options.index(correct_answer)

  try:
    model_output_json = json.loads(model_output)
    model_pred_option = model_output_json['answer_letter'].lower()

    if model_pred_option == 'abcd'[correct_answer_index]:
      return True
    else:
      return False
  except:
    matches = [options[i].lower() in model_output.lower() for i in range(4)]

    # matches has 4 elements, eg. for correct model response for correct answer being A: [True, False, False, False]
    # Here, we will flip only the value at the correct index i.e. it should go from True to False
    # If any other option matches with the model output or if the correct one doesn't match, then we return False,
    # If everything is exactly how it should be (model response is correct), then we return True
    matches[correct_answer_index] = not matches[correct_answer_index]

    # print(f'\n\n\nModel Output: \n{model_output}\n\nOptions: \n{options}\n\nCorrect Answer: {correct_answer}\n\nMatches List: {matches}')

    for i in matches:
      if i:
        return False
    return True





def calculate_accuracy(model_outputs, context_question_choice_combos, correct_answers):
  """

  Args:
    model_outputs: List <Str>
    context_question_choice_combos: List <Dict>

  Returns: Accuracy (# Correct / # Total)
  """

  count = 0
  n = len(model_outputs)
  for ind in range(n):
    model_output = list(model_outputs[ind])[0]
    # print(model_output, type(model_output))
    options = context_question_choice_combos[ind]['Options']
    correct_answer = correct_answers[ind]

    if check_match(model_output=model_output, options=options, correct_answer=correct_answer):
      count += 1

  return count/n

In [ ]:
calculate_accuracy(model_outputs_relevant, correct_context_question_choice_combos, correct_answers)

In [ ]:
model_outputs_irrelevant = run_method_relevant_context(incorrect_context_question_choice_combos[:20])
model_outputs_irrelevant

100%|██████████| 20/20 [00:43<00:00,  2.15s/it]


[{'{"thought": "Let us think step by step. The context provided discusses a genetic study conducted on mummified baboons. This is unrelated to the question about the apparition. The Virgin Mary allegedly appeared to Saint Bernadette Soubirous in 1858 in Lourdes, France. It was Saint Bernadette Soubirous who claimed to have seen the apparitions. Therefore, the correct answer is: B. Saint Bernadette Soubirous.", "answer_letter": "B", "answer_name": "Saint Bernadette Soubirous"}"}'},
 {'{"thought": "Let us think step by step. The context does not mention anything about the Scholastic Magazine of Notre Dame, so we cannot determine the publishing date based on the given information.", "answer_letter": "B", "answer_name": "January 1872"}'},
 {'{"thought": "Let us think step by step. The context provided does not mention the headquarters of the Congregation of the Holy Cross. Therefore, we need to rely on prior knowledge or an external source to answer this question.", "answer_letter": "B", "

In [ ]:
# Irrelevant Context Method Accuracy
calculate_accuracy(model_outputs_irrelevant, incorrect_context_question_choice_combos, correct_answers)

In [ ]:
# No context accuracy.
model_outputs_no_context = run_method_relevant_context(correct_context_question_choice_combos[:20], method = 'no_context')
model_outputs_no_context

100%|██████████| 20/20 [00:42<00:00,  2.12s/it]


[{'{"thought": "Let us think step by step. The context is missing, but commonly, it is known that the Virgin Mary allegedly appeared to Saint Bernadette Soubirous in 1858 in Lourdes, France. Therefore, the correct answer is: B. Saint Bernadette Soubirous.", "answer_letter": "B", "answer_name": "Saint Bernadette Soubirous"}"'},
 {'{"thought": "Let us think step by step. There is no specific information provided in the context about the Scholastic Magazine of Notre Dame. We need to provide the best guess based on the probable time period the Scholastic Magazine of Notre Dame might have begun publishing. Since January 1872 is the earliest option, let\'s choose that as a possible starting point.", "answer_letter": "B", "answer_name": "January 1872"}'},
 {'{"thought": "Let us think step by step. Without additional information provided in the context, we can recall that the headquarters of the Congregation of the Holy Cross is traditionally located in Rome, Italy. The Congregation of the Hol

In [ ]:
# No Context Method Accuracy
calculate_accuracy(model_outputs_no_context, correct_context_question_choice_combos, correct_answers)